In [ ]:
import dask.array as da
import numpy as np
import dask
from dask import delayed
from toolz import partition
from tqdm import tqdm
dask.config.set({'distributed.comm.timeouts.connect':'160s'})

In [2]:
source_file = '/groups/hess/hesslab/HighResPaper_rawdata/9. TCell_on_cancer_4x4x4nm/TCell_on_cancer_Atlas1_4x4x4nm.mrc'

In [3]:
def minmax(sl):
    fname = source_file
    from mrcfile.mrcmemmap import MrcMemmap
    arr = MrcMemmap(fname).data[sl]
    return arr.min(), arr.max()

def mrc_to_dask(fname, stride):
    from mrcfile.mrcmemmap import MrcMemmap
    from itertools import product
    with MrcMemmap(fname) as mem:
        shape = mem.data.shape
        dtype = mem.data.dtype
        if mem.data.flags['C_CONTIGUOUS']:
            chunks = (stride, -1,-1)
            concat_axis = 0
        elif mem.data.flags['F_CONTIGUOUS']:
            chunks = (-1,-1, stride)
            concat_axis = len(shape) - 1
        else:
            raise ValueError('Could not infer whether array is C or F contiguous')
    template = da.zeros(shape=shape, dtype=dtype, chunks=chunks)    
    slices = da.core.slices_from_chunks(template.chunks)
    shapes = list(product(*template.chunks))
    @delayed
    def loader(sl): 
        import mrcfile.mrcmemmap as mm
        import numpy as np
        with mm.MrcMemmap(fname) as mrc:
            result = np.array(mrc.data[sl])
        return result
    
    arr = da.concatenate([da.from_delayed(loader(sl), shape=shapes[idx], dtype=dtype) for idx,sl in enumerate(slices)], axis=concat_axis)
    return arr

In [4]:
sample = mrc_to_dask(source_file, 10)

In [5]:
from fst.distributed import get_cluster
cl = get_cluster(walltime="300")
cl

Client Scheduler: tcp://10.36.110.11:15222 Dashboard: http://10.36.110.11:8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [7]:
%%time
cl.cluster.scale(60)
result = cl.gather(cl.compute([sample.min(), sample.max()]))
cl.cluster.scale(0)

CPU times: user 32.4 s, sys: 1.67 s, total: 34 s
Wall time: 3min 27s


In [20]:
%%time
cl.cluster.scale(60)
result = sample.map_blocks(lambda v: np.expand_dims(np.array([v.min(), v.max()]), (0,1))).compute(scheduler=cl)
cl.cluster.scale(0)

CPU times: user 16.4 s, sys: 1.22 s, total: 17.6 s
Wall time: 1min 59s
